In [9]:
import pickle
import pandas as pd
import plotly.offline as pyo
import plotly.express as px
import dimensionality_reduction
import json 
import random
from numpy import dot
from numpy.linalg import norm
import numpy as np
from utils import aggregate_embeddings, cosine_similarity, print_similarity_samples, replace_nan_with, load_model, load_big_consulting_export, split_into_list
from tqdm import tqdm

tqdm.pandas()


In [10]:
big_consulting_df = load_big_consulting_export()
data_list = []

for index, row in tqdm(big_consulting_df.iterrows()):
    context = row['snippet']
    
    qas_list = []
    
    def get_qas_item(question, company):
        answer_found = False
        if company is None:
            return {
                'id': str(index).zfill(5),
                'is_impossible': True,
                'question': question,
                'answers': []
            }
        company_words = company.split()

        for i in range(len(company_words), 0, -1):
            company_part = ' '.join(company_words[:i])
            if company_part in context:
                answer_found = True
                answer_text = company_part
    #             answer_start = context.find(answer_text)
                break

        if not answer_found:
            answer_text = company_words[0]
        answer_start = context.find(answer_text)

        qas_item = {
            'id': str(index).zfill(5),
            'is_impossible': False,
            'question': question,
            'answers': [{
                'text': answer_text,
                'answer_start': answer_start
            }]
        }
        
        
        return qas_item
    
        # If 'relationEntityType' is 'company', remove the value in 'relationEntity'
        df.at[index, 'relationEntity'] = None


    qas_item = get_qas_item("What is the company's name?", row['company'])
    if row['relationEntityType'] == 'company' and not pd.isnull(row['relationEntity']):
        qas_item_2 = get_qas_item("What is the other company's name?", row['relationEntity'])
    else:
        qas_item_2 = get_qas_item("What is the other company's name?", None)
    qas_list.append(qas_item)
    qas_list.append(qas_item_2)
        
    data_item = {
        'context': context,
        'qas': qas_list
    }
    
    data_list.append(data_item)

data_list[:10]

12063it [00:00, 41279.68it/s]


[{'context': 'at KPMG where he focused on complex financial matters and financial trading, investment banking and lending businesses',
  'qas': [{'id': '00000',
    'is_impossible': False,
    'question': "What is the company's name?",
    'answers': [{'text': 'KPMG', 'answer_start': 3}]},
   {'id': '00000',
    'is_impossible': True,
    'question': "What is the other company's name?",
    'answers': []}]},
 {'context': 'LatentView has been recognized as an industry leader by Gartner',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': "What is the company's name?",
    'answers': [{'text': 'Gartner', 'answer_start': 56}]},
   {'id': '00001',
    'is_impossible': True,
    'question': "What is the other company's name?",
    'answers': []}]},
 {'context': 'In his last role, Prashant was playing the role of Portfolio & Regional Head at Infosys, as a key leader for their Communications, Media and Entertainment business',
  'qas': [{'id': '00002',
    'is_impossible': 

In [14]:
random.seed(42)
random.shuffle(data_list)
test_data = data_list[:100]
val_data = data_list[100:200]
train_data = data_list[200:]

In [15]:
import json
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
 
model_type="bert"
model_name= "bert-base-cased"
 
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
#     "output_dir": f"outputs/{model_type}",
#     "best_model_dir": f"{output_dir}/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 2,
    "evaluate_during_training_steps": 100,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":8,
    "train_batch_size": 16,
    "eval_batch_size": 16
}
model = QuestionAnsweringModel(model_type,model_name, args=train_args, use_cuda=False)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# https://thinkinfi.com/fine-tune-bert-for-question-answering/
model.train_model(train_data, eval_data=test_data)
result, texts = model.eval_model(test_data)

convert squad examples to features:   0%| | 0Could not find answer: '' vs. 'America'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Fnts'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'Gartner'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'Aon'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Gartner'
Could not find answer: '' vs. 'Wipro'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'PricewaterhouseCoop

Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Globalfoundries'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'Ernst'


Could not find answer: '' vs. 'Gartner'
Could not find answer: '' vs. 'AlixPartners'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'Globalfoundries'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'Ernst'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'Stepzen'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Globalfou

Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'Aon'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Hcltech'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Ernst'
Could not find answer: '' vs. 'Ernst'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Ernst'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'Pricewater

Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Ernst'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'Ernst'
Could not find answer: '' vs. 'Infosys'
Could not find answer: '' vs. 'E&Y'
Could not find answer: '' vs. 'Fti'
Could not find answer: '' vs. 'Ernst'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Ernst'
Could not find answer: '' vs. 'AlixPartners'
Could not find answer: '' vs. 'Tata'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'Bain'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'IBM'
Could not find answer: '' vs. 'PricewaterhouseCoopers'
Could not find answer: '' vs. 'Ernst'
Could not find answer: '' vs. 'Ta

...

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [17]:
result, texts

({'correct': 90,
  'similar': 9,
  'incorrect': 1,
  'eval_loss': -7.623627809377817},
 {'correct_text': {'04165': '',
   '05680': '',
   '08171': '',
   '07163': '',
   '05207': '',
   '13343': '',
   '10926': '',
   '11356': '',
   '10008': '',
   '11150': '',
   '02421': '',
   '13115': '',
   '06551': '',
   '09305': '',
   '06138': '',
   '01814': '',
   '11872': '',
   '00196': '',
   '11553': '',
   '05293': '',
   '08185': '',
   '04640': '',
   '13350': '',
   '07424': '',
   '08832': '',
   '00238': '',
   '10940': '',
   '04952': '',
   '07046': 'Threadgill Financial LLC',
   '10653': '',
   '04653': '',
   '09520': '',
   '07879': '',
   '12744': '',
   '11402': 'QPR Software',
   '03979': '',
   '04821': '',
   '11664': '',
   '06939': '',
   '09433': '',
   '01889': '',
   '08702': '',
   '03244': '',
   '11501': '',
   '09493': '',
   '05274': '',
   '11981': '',
   '10096': '',
   '11574': '',
   '04968': '',
   '12465': '',
   '11021': '',
   '12859': '',
   '09284': '

In [ ]:
test_data

In [55]:
import pandas as pd

# Read the original file
input_file_path = '../glanos-data/clustering/big_consulting_export_replace.tsv'
df = pd.read_csv(input_file_path, sep='\t')

# Drop duplicates based on the 'replace' column
df_no_duplicates = df.drop_duplicates(subset='replace')

# Save the DataFrame to a new file
output_file_path = '../glanos-data/clustering/big_consulting_export_replace_no_duplicates.tsv'
df_no_duplicates.to_csv(output_file_path, sep='\t', index=False)
